## Imports

In [61]:
import fiona
import geopandas as gpd
import folium
import os

## Read in data

In [62]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
greater_dtown_miami = gpd.read_file('Greater Downtown Miami.kml', driver = 'KML')

In [63]:
miami_dade_permits = gpd.read_file('17e7aee8-0a8b-4ef8-9464-5575d5550939.gdb/')

## Filter to only Greater Miami Area

In [64]:
# Perform a spatial join
joined_permits = gpd.sjoin(miami_dade_permits, greater_dtown_miami, how='inner', op='within')

/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3309: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [65]:
print(len(miami_dade_permits))
print('-------')
print(len(joined_permits))

275860
-------
318


In [66]:
joined_permits.columns

Index(['ID', 'ADDRESS', 'STNDADDR', 'UNIT', 'PROCNUM', 'FOLIO', 'GEOFOLIO',
       'ISCONDO', 'TYPE', 'CAT1', 'DESC1', 'CAT2', 'DESC2', 'CAT3', 'DESC3',
       'CAT4', 'DESC4', 'CAT5', 'DESC5', 'CAT6', 'DESC6', 'CAT7', 'DESC7',
       'CAT8', 'DESC8', 'CAT9', 'DESC9', 'CAT10', 'DESC10', 'ISSUDATE',
       'LSTINSDT', 'RENDATE', 'CCDATE', 'BLDCMPDT', 'RESCOMM', 'BPSTATUS',
       'PROPUSE', 'APPTYPE', 'CLUC', 'FFRMLINE', 'LGLDESC1', 'LGLDESC2',
       'ESTVALUE', 'PTSOURCE', 'MPRMTNUM', 'LSTAPPRDT', 'CONTRNUM',
       'CONTRNAME', 'GlobalID', 'geometry', 'index_right', 'Name',
       'Description'],
      dtype='object')

In [67]:
def popup_html(row):
    address = row['ADDRESS']
    sdn_address = row['STNDADDR']
    folio = row['FOLIO']
    type_ = row['TYPE']
    issue_date = row['ISSUDATE']
    geometry = row['geometry']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Standard Address: </strong>{}'''.format(sdn_address) + '''<br>
    <strong>Folio: </strong>{}'''.format(folio) + '''<br>
    <strong>Type: </strong>{}'''.format(type_) + '''<br>
    <strong>Issue Date: </strong>{}'''.format(issue_date) + '''<br>
    <strong>Location: </strong>${}'''.format(geometry) + '''<br>
    </html>
    '''
    return html

In [68]:
# Create a map centered on Miami
m = folium.Map(location=[25.7617, -80.1918], zoom_start=12)

for i, r in greater_dtown_miami.iterrows():
    try:
      sim_geo = gpd.GeoSeries(r['geometry'])
      geo_j = sim_geo.to_json()
      geo_j = folium.GeoJson(data=geo_j)
      folium.Tooltip(r['Name']).add_to(geo_j)
      geo_j.add_to(m)
    except Exception as e:
      print(e)
      pass

# Add the points to the map
for index, row in joined_permits.iterrows():
    folium.Marker(location=[row['geometry'].y, row['geometry'].x],popup=folium.Popup(popup_html(row),max_width=400)).add_to(m)

# Display the map
m

In [69]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [70]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Greater_Miami_Projects


In [71]:
joined_permits.to_csv('Greater_Dtown_Miami_Permits.csv')